In [1]:
import pandas as pd
import numpy as np
import re
import gensim
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from wordcloud import WordCloud
from gensim import corpora, models, similarities
import pyLDAvis
from pprint import pprint
import pickle 
import os
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
import random
from gensim.models.ldamodel import LdaModel
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import MmCorpus
import csv
import pyLDAvis.gensim_models as gensimvis
RANDOM_STATE = 1

In [2]:
data = pd.read_csv('dataset/DatasetLegal.csv')
with open('model/id2word.pkl', 'rb') as f:
    id2word = pickle.load(f)
with open("lda/topic_dict_5.pkl", "rb") as f:
   topic_dict = pickle.load(f)
with open("lda/lda_model_5.pkl", "rb") as f:
    lda_model = pickle.load(f)
corpus_question = corpora.MmCorpus('model/corpus_question.mm')

str_answer = data['answer'].astype(str)
str_answer = str_answer.map(lambda x: re.sub('[,.!?*#/]', '', x))
str_question = data['question'].astype(str)
str_question = str_question.map(lambda x: re.sub('[,.!?*#/]', '', x))

train_answer = list(str_answer[:5625])
test_answer = list(str_answer[5625:])
train_question = list(str_question[:5625])
test_question = list(str_question[5625:])

In [21]:
label = data['Classes']
label_train = label[:5625]
label_test = label[5625:]
label_train = label_train.values.tolist()
label_test = label_test.values.tolist()
se_topic = []
family = []
contract = []
criminal = []
Violation = []
labor = []
for i in range(len(label_train)):
    if(label_train[i] == 'family'): 
        family.append(i)
    elif(label_train[i] == 'contract'): 
        contract.append(i)
    elif(label_train[i] == 'criminal'): 
        criminal.append(i)
    elif(label_train[i] == 'Violation' or label_train[i] == 'violation'): 
        Violation.append(i)
    elif(label_train[i] == 'labor'): 
        labor.append(i)
se_topic.append(family)
se_topic.append(contract)
se_topic.append(criminal)
se_topic.append(Violation)
se_topic.append(labor)
se_topic_test = []
family_test = []
contract_test = []
criminal_test = []
Violation_test = []
labor_test = []
for i in range(len(label_test)):
    if(label_test[i] == 'family'): 
        family_test.append(i)
    elif(label_test[i] == 'contract'): 
        contract_test.append(i)
    elif(label_test[i] == 'criminal'): 
        criminal_test.append(i)
    elif(label_test[i] == 'Violation' or label_test[i] == 'violation'): 
        Violation_test.append(i)
    elif(label_test[i] == 'labor'): 
        labor_test.append(i)
se_topic_test.append(family_test)
se_topic_test.append(contract_test)
se_topic_test.append(criminal_test)
se_topic_test.append(Violation_test)
se_topic_test.append(labor_test)
# len(se_topic_test)

In [26]:
corpus_question

In [4]:
def preprocess(text):
    stopwords = list(thai_stopwords())
    read_stopwords = pd.read_csv('dataset/add_stopwords.csv')
    add_stopwords = read_stopwords['stopword'].values.tolist()
    result = []
    str_text = str(text).replace(' ','')
    word_token = word_tokenize(str_text, engine='newmm')
    for word in word_token:
        if(word not in stopwords + add_stopwords):
            result.append(word)
    return result

In [5]:
corpus_lda = lda_model[corpus_question]
index = similarities.MatrixSimilarity(corpus_lda, num_features=len(id2word))
def lda_sim(new_doc_topics):
    sims = index[new_doc_topics]
    sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
    return sims_sorted[:100]

In [6]:
similarity_index = similarities.MatrixSimilarity(corpus_question)
def bow_sim(new_doc_topics):
    similarities = similarity_index[new_doc_topics]
    sorted_similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
    return sorted_similarities[:100]

family

In [60]:
bow_family_accuracy = []
for i in se_topic_test[0]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    topic = bow_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[0]):
            count_true += 1
        else:
            count_false += 1
    bow_family_accuracy.append((count_true)/(count_true+count_false))
bow_family_accuracy_result = sum(bow_family_accuracy)/len(bow_family_accuracy)
bow_family_accuracy_result

0.8534515366430272

In [19]:
family_accuracy = []
for i in se_topic_test[0]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[0]):
            count_true += 1
        else:
            count_false += 1
    family_accuracy.append((count_true)/(count_true+count_false))
family_accuracy_result = sum(family_accuracy)/len(family_accuracy)

In [46]:
family_accuracy_result

0.9243498817966926

In [7]:
import random
n = random.randint(1, 1406)

random_family_accuracy = []
for i in range(100):
    new_doc_topics = preprocess(test_question[random.randint(1, 1406)])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[0]):
            count_true += 1
        else:
            count_false += 1
    random_family_accuracy.append((count_true)/(count_true+count_false))
random_family_accuracy_result = sum(random_family_accuracy)/len(random_family_accuracy)


In [66]:
random_family_accuracy_result

0.35010000000000013

In [ ]:
random_family_accuracy_result

contract

In [8]:
bow_contract_accuracy = []
for i in se_topic_test[1]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    topic = bow_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[1]):
            count_true += 1
        else:
            count_false += 1
    bow_contract_accuracy.append((count_true)/(count_true+count_false))
bow_contract_accuracy_result = sum(bow_contract_accuracy)/len(bow_contract_accuracy)
bow_contract_accuracy_result

0.6789161554192236

In [22]:
contract_accuracy = []
for i in se_topic_test[1]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[1]):
            count_true += 1
        else:
            count_false += 1
    contract_accuracy.append((count_true)/(count_true+count_false))
contract_accuracy_result = sum(contract_accuracy)/len(contract_accuracy)

In [71]:
contract_accuracy_result

0.6870552147239257

In [72]:
random_contract_accuracy = []
for i in range(100):
    new_doc_topics = preprocess(test_question[random.randint(1, 1406)])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[1]):
            count_true += 1
        else:
            count_false += 1
    random_contract_accuracy.append((count_true)/(count_true+count_false))
random_contract_accuracy_result = sum(random_contract_accuracy)/len(random_contract_accuracy)

In [73]:
random_contract_accuracy_result

0.3760000000000001

criminal

In [14]:
bow_criminal_accuracy = []
for i in se_topic_test[2]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    topic = bow_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[2]):
            count_true += 1
        else:
            count_false += 1
    bow_criminal_accuracy.append((count_true)/(count_true+count_false))
bow_criminal_accuracy_result = sum(bow_criminal_accuracy)/len(bow_criminal_accuracy)
bow_criminal_accuracy_result

0.6011009174311929

In [23]:
criminal_accuracy = []
for i in se_topic_test[2]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[2]):
            count_true += 1
        else:
            count_false += 1
    criminal_accuracy.append((count_true)/(count_true+count_false))
criminal_accuracy_result = sum(criminal_accuracy)/len(criminal_accuracy)

In [56]:
criminal_accuracy_result

0.541880733944954

In [24]:
random_criminal_accuracy = []
for i in range(100):
    new_doc_topics = preprocess(test_question[random.randint(1, 1406)])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[2]):
            count_true += 1
        else:
            count_false += 1
    random_criminal_accuracy.append((count_true)/(count_true+count_false))
random_criminal_accuracy_result = sum(random_criminal_accuracy)/len(random_criminal_accuracy)

In [75]:
random_criminal_accuracy_result

0.16179999999999997

Violation

In [22]:
bow_violation_accuracy = []
for i in se_topic_test[3]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    topic = bow_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[3]):
            count_true += 1
        else:
            count_false += 1
    bow_violation_accuracy.append((count_true)/(count_true+count_false))
bow_violation_accuracy_result = sum(bow_violation_accuracy)/len(bow_violation_accuracy)
bow_violation_accuracy_result

0.12173469387755104

In [18]:
Violation_accuracy = []
for i in se_topic_test[3]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[3]):
            count_true += 1
        else:
            count_false += 1
    Violation_accuracy.append((count_true)/(count_true+count_false))
Violation_accuracy_result = sum(Violation_accuracy)/len(Violation_accuracy)
Violation_accuracy_result

0.14326530612244903

In [12]:
random_Violation_accuracy = []
for i in range(100):
    new_doc_topics = preprocess(test_question[random.randint(1, 1406)])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[3]):
            count_true += 1
        else:
            count_false += 1
    random_Violation_accuracy.append((count_true)/(count_true+count_false))
random_Violation_accuracy_result = sum(random_Violation_accuracy)/len(random_Violation_accuracy)

In [13]:
random_Violation_accuracy_result

0.04419999999999996

labor

In [27]:
bow_labor_accuracy = []
for i in se_topic_test[4]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    topic = bow_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[4]):
            count_true += 1
        else:
            count_false += 1
    bow_labor_accuracy.append((count_true)/(count_true+count_false))
bow_labor_accuracy_result = sum(bow_labor_accuracy)/len(bow_labor_accuracy)
bow_labor_accuracy_result

0.7725280898876408

In [28]:
labor_accuracy = []
for i in se_topic_test[4]:
    new_doc_topics = preprocess(test_question[i])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[4]):
            count_true += 1
        else:
            count_false += 1
    labor_accuracy.append((count_true)/(count_true+count_false))
labor_accuracy_result = sum(labor_accuracy)/len(labor_accuracy)

In [61]:
labor_accuracy_result

0.807752808988764

In [78]:
random_labor_accuracy = []
for i in range(100):
    new_doc_topics = preprocess(test_question[random.randint(1, 1406)])
    new_doc_topics = id2word.doc2bow(new_doc_topics)
    new_doc_topics = lda_model.get_document_topics(new_doc_topics)
    topic = lda_sim(new_doc_topics)
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[4]):
            count_true += 1
        else:
            count_false += 1
    random_labor_accuracy.append((count_true)/(count_true+count_false))
random_labor_accuracy_result = sum(random_labor_accuracy)/len(random_labor_accuracy)

In [79]:
random_labor_accuracy_result

0.15249999999999997

average

In [16]:
bow_average = average = (bow_labor_accuracy_result+bow_criminal_accuracy_result+bow_contract_accuracy_result+bow_family_accuracy_result+bow_Violation_accuracy_result)/5
bow_average

0.605546278651727

In [63]:
average = (labor_accuracy_result+criminal_accuracy_result+contract_accuracy_result+family_accuracy_result+Violation_accuracy_result)/5
average

0.6206200228702088

In [18]:
10*average - 10*bow_average

0.0

In [81]:
random_average = (random_labor_accuracy_result+random_criminal_accuracy_result+random_contract_accuracy_result+random_family_accuracy_result+random_Violation_accuracy_result)/5
random_average

0.21682000000000007

In [82]:
average - random_average

0.4038000228702087

apply keyword based + topic based

In [82]:
new_doc = data['question'][1]
new_doc

'#บิดาดิฉันมีลูกติดคนมารดามีลูกติดคน#บิดามารดาสมรสมีบุตรคือตัวดิฉัน#จดทะเบียนสมรสกันหลังจากดิฉันโต#มีโฉนดที่ดินชื่อบิดามารดาร่วมกัน#บิดาเสียชีวิตมารดายังมีชีวิตแต่มารดาแก่มากหลงๆลืม#ดิฉันอยากทำเป็นผู้จัดการมรดกเอาชื่อบิดามารดาออกเป็นชื่อดิฉันคนเดียว#คำถามลูกติดทั้งคนต้องเซ็นเอกสารอะไรไหมคะเพราะคนนึงอยู่ประจวบคนนึงอยู่หนองคายแต่ละคนไม่สะดวกมาคะชอบพระคุณล่วงหน้านะคะ'

In [69]:
corpus_lda = lda_model[corpus_question]
index_lda = similarities.MatrixSimilarity(corpus_lda, num_features=len(id2word))
index_bow = similarities.MatrixSimilarity(corpus_question)

In [107]:
def topic_keyword_based(new_doc):
    new_doc_topics = id2word.doc2bow(preprocess(new_doc))
    lda_topics = lda_model.get_document_topics(new_doc_topics)
    sims_lda = index_lda[lda_topics]
    sims_bow = index_bow[new_doc_topics]
    max_lda = max(sims_lda)
    max_bow = max(sims_bow)
    weight = max_lda/max_bow
    sum = sims_lda+(sims_bow*weight)
    sims_sorted = sorted(enumerate(sum), key=lambda item: -item[1])
    return sims_sorted

In [108]:
mix_family_accuracy = []
for i in se_topic_test[0]:
    topic = topic_keyword_based(test_question[i])
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[0]):
            count_true += 1
        else:
            count_false += 1
    mix_family_accuracy.append((count_true)/(count_true+count_false))
mix_family_accuracy_result = sum(mix_family_accuracy)/len(mix_family_accuracy)
mix_family_accuracy_result

0.9414657210401902

In [110]:
mix_contract_accuracy = []
for i in se_topic_test[1]:
    topic = topic_keyword_based(test_question[i])
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[1]):
            count_true += 1
        else:
            count_false += 1
    mix_contract_accuracy.append((count_true)/(count_true+count_false))
mix_contract_accuracy_result = sum(mix_contract_accuracy)/len(mix_contract_accuracy)
mix_contract_accuracy_result

0.786646216768917

In [111]:
mix_criminal_accuracy = []
for i in se_topic_test[2]:
    topic = topic_keyword_based(test_question[i])
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[2]):
            count_true += 1
        else:
            count_false += 1
    mix_criminal_accuracy.append((count_true)/(count_true+count_false))
mix_criminal_accuracy_result = sum(mix_criminal_accuracy)/len(mix_criminal_accuracy)
mix_criminal_accuracy_result

0.7541743119266053

In [112]:
mix_Violation_accuracy = []
for i in se_topic_test[3]:
    topic = topic_keyword_based(test_question[i])
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[3]):
            count_true += 1
        else:
            count_false += 1
    mix_Violation_accuracy.append((count_true)/(count_true+count_false))
mix_Violation_accuracy_result = sum(mix_Violation_accuracy)/len(mix_Violation_accuracy)
mix_Violation_accuracy_result

0.19142857142857142

In [113]:
mix_labor_accuracy = []
for i in se_topic_test[4]:
    topic = topic_keyword_based(test_question[i])
    count_true = 0
    count_false = 0
    for j in range (100):
        if(topic[j][0] in se_topic[4]):
            count_true += 1
        else:
            count_false += 1
    mix_labor_accuracy.append((count_true)/(count_true+count_false))
mix_labor_accuracy_result = sum(mix_labor_accuracy)/len(mix_labor_accuracy)
mix_labor_accuracy_result

0.8919101123595502

In [115]:
mix_average = mix_family_accuracy_result+mix_criminal_accuracy_result+mix_contract_accuracy_result+mix_labor_accuracy_result+mix_Violation_accuracy_result
mix_average = mix_average/5
mix_average
# better than lda 9.252

0.7131249867047668